In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import Nystroem
from sklearn import metrics
from sklearn.svm import OneClassSVM
import time
import copy

In [23]:
import re
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import os,sys
import tqdm
import pickle
import time
from concurrent.futures import ThreadPoolExecutor  
import concurrent.futures

In [24]:
def reshape_matrix(matrix_list):
    new_list = [np.array(i).reshape(-1) for i in matrix_list]
    return new_list

In [25]:
def padding_onehot(onehot_list, padding):
    new_list = []
    for onehot in onehot_list:
        if len(onehot) > padding:
            onehot = np.array(onehot[0:padding])
            new_list.append(onehot)
        else:
            onehot =np.pad(onehot, [(0, padding-len(onehot)), (0, 0)], mode='constant', constant_values=0)
            new_list.append(onehot)
    new_list = reshape_matrix(new_list)
    return new_list
    

In [26]:
def padding_dictencoding(dictencoding_list, padding):
    new_list = []
    for onehot in dictencoding_list:
        if len(onehot) > padding:
            onehot = np.array(onehot[0:padding])
            new_list.append(onehot)
        else:
            onehot =np.pad(onehot, [(0, padding-len(onehot))], mode='constant', constant_values=0)
            new_list.append(onehot)
    return new_list

In [27]:
def train_models(feature, normal):
    X = normal[feature].tolist()
    y = normal['maltype'].tolist()
    # mlb = LabelBinarizer()
    
    y = np.zeros(len(X))
    # h = .02  # step size in the mesh
    outliers_fraction = 0.15
    nu = 0.05
    clfs = []
    results = []
    preds = []
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3, random_state=42)
    y_val_2 = [1 if i==0 else -1 for i in y_val]

    X_tv = []
    pca = PCA(n_components=100)
    if feature =='system calls dependency graph':
        X_train = reshape_matrix(X_train)
        X_val = reshape_matrix(X_val)    
        X_tv=[(X_train,X_val)]
        features = [feature]
    elif feature =='one hot encoding':
        X_train_55000 = padding_onehot(X_train, 55000)
        X_val_55000 = padding_onehot(X_val, 55000)
        X_train_pca = pca.fit_transform(X_train_55000)
        X_val_pca = pca.transform(X_val_55000)
        X_tv = [(X_train_pca,X_val_pca)]
        features = [feature]
    elif feature =='dict index encoding':
        X_train_55000 = padding_dictencoding(X_train, 55000)
        X_val_55000 = padding_dictencoding(X_val, 55000)
        X_train_pca = pca.fit_transform(X_train_55000)
        X_val_pca = pca.transform(X_val_55000)
        X_train_10000 = padding_dictencoding(X_train, 10000)
        X_val_10000 = padding_dictencoding(X_val, 10000)
        X_tv = [(X_train_pca,X_val_pca),(X_train_10000,X_val_10000)]
        features = [feature+'-pca', feature]
    elif 'system calls hashing' in feature or '1gram'in feature:
        X_tv=[(X_train,X_val)]
        features = [feature]
    else:
        X_train_pca = pca.fit_transform(X_train)
        X_val_pca = pca.transform(X_val)
        X_tv = [(X_train_pca,X_val_pca),(X_train,X_val)]
        features = [feature+'-pca', feature]

    for i,(X_train,X_val)  in enumerate(X_tv):
        feature = features[i]
        result = []
        pred = dict()
        classifiers = {
                "Robust covariance": EllipticEnvelope(contamination=outliers_fraction),
                "One-Class SVM": OneClassSVM(nu=outliers_fraction, kernel="rbf",gamma=0.1),
                "SGD One-Class SVM": SGDOneClassSVM(nu=nu, shuffle=True, fit_intercept=True, random_state=42, tol=1e-4),
                "Isolation Forest": IsolationForest(contamination=outliers_fraction,random_state=42),
            }
        for name in classifiers:
            clf = classifiers[name]
            t1 =time.time()
            res = dict()
            clf.fit(X_train)
            t2 =time.time()
            y_pred = clf.predict(X_val)
            score = metrics.accuracy_score(y_val_2,y_pred)

            pred['valid_'+  feature + '_' + name] = y_pred
            t = t2 -t1
            res['Model'] ='valid_' + feature + '_' + name
            res['Accuracy'] = score
            res['Training time'] = t
            result.append(res)
            print('Model: {}, accuracy score: {}, training time is: {} seconds'.format(res['Model'], score, t))
        results.append(result)
        preds.append(pred)
        clfs.append(classifiers)
    pca = pca
    return clfs, results, preds, pca

In [28]:
def test_models(encoded_trace_df, malware, feature, clfs, pca):
    dfs = encoded_trace_df[encoded_trace_df.maltype==malware]
    X_test = dfs[feature].tolist()
    X_tv = [] 
    if feature =='system calls dependency graph':
        X_test = reshape_matrix(X_test)    
        X_tv=[X_test]
        features = [feature]
    elif feature =='one hot encoding':
        X_test_55000 = padding_onehot(X_test, 55000)
        X_test_pca = pca.transform(X_test_55000)
        X_tv = [X_test_pca]
        features = [feature]
    elif feature =='dict index encoding':
        X_test_55000 = padding_dictencoding(X_test, 55000)       
        X_test_pca = pca.transform(X_test_55000)
        X_test_10000 = padding_dictencoding(X_test, 10000)       
        X_tv = [X_test_pca, X_test_10000]
        features = [feature+'-pca', feature]
    elif 'system calls hashing' in feature or '1gram'in feature:
        X_tv=[X_test]
        features = [feature]
    else:

        X_test_pca = pca.transform(X_test)
        X_tv = [X_test_pca, X_test]
        features = [feature+'-pca', feature]

    results = []
    preds = []
    y_test = np.ones(len(X_test))
    y_test_2 = [1 if i==0 else -1 for i in y_test]

    for i,classifiers in enumerate(clfs):
        feature = features[i]
        X_test = X_tv[i]
        result = []
        pred = dict()    
        for name in classifiers:
            res = dict()
            clf = classifiers[name]
            t1 =time.time()
            if 'anomaly detection' in name:
                y_pred = clf.predict(X_test)
                t2 =time.time()
                score = metrics.accuracy_score(y_test_2,y_pred)
            else:
                y_pred = clf.predict(X_test)
                t2 =time.time()
                score = metrics.accuracy_score(y_test,y_pred)
            t = t2 -t1
            pred[malware +'_' + feature + '_' + name] = y_pred
            res['Model'] = malware +'_' + feature + '_' + name
            res['Accuracy'] = score
            res['Testing time'] = t
            result.append(res)
            print('Model: {}, accuracy score: {}, testing time is: {} seconds'.format( res['Model'], score, t))
        results.append(result)
        preds.append(pred)
    return results, preds       

In [29]:
rootPath ='D:/git/IoT_Sensors_Security_Analysis/data/perf/'
encodePath = rootPath+'encoded/t1/'

In [30]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [31]:
def run(device, tw):
    resultsPath = 'D:/git/IoT_Sensors_Security_Analysis/results/{}/tw_{}_turn_1/'.format(device, tw)
    encoded_trace_df = pd.read_pickle(encodePath+'encoded_bow{}_{}.pkl'.format(device, tw))
    normal = encoded_trace_df[encoded_trace_df.maltype=='normal']
    resultsdict = dict()
    predsdict = dict()
    classifiersdict = dict()
    pcas = dict()
    for feature in features:
        #train stage
        clfs, results, preds, pca = train_models(feature, normal)
        
        resultsdict[feature+'_validation'] = results
        predsdict[feature+'_validation'] = preds
        classifiersdict[feature] = clfs
        pcas[feature] = pca
        # testing stage
        for malware in malwares:
            results, preds = test_models(encoded_trace_df, malware, feature, clfs, pca)
            resultsdict[malware +'_' + feature] = results
            predsdict[malware +'_' + feature] = preds
            
    loc=open(resultsPath+'classifiers.pk','wb')
    pickle.dump(classifiersdict,loc)
    loc=open(resultsPath+'results.pk','wb')
    pickle.dump(resultsdict,loc)
    loc=open(resultsPath+'preds.pk','wb')
    pickle.dump(predsdict,loc)   
    loc=open(resultsPath+'pcas.pk','wb')
    pickle.dump(pcas,loc)   
    rd = []
    for rs in resultsdict:
        for r in resultsdict[rs]:
            for s in r:
                rd.append(s)

    rd = pd.DataFrame(rd)
    md = [i.split('_') for i in rd['Model']]
    md = pd.DataFrame(md)
    md.columns  = ['Dataset','Features','Ngram','Model']
    nrd=pd.DataFrame([md['Dataset'],md['Features'],md['Ngram'], md['Model'], rd['Accuracy']]).transpose()
    nrd.to_csv(resultsPath+'results.csv',index=None)

In [32]:
features = [#'one hot encoding', 'dict index encoding',
            'system calls dependency graph', 'system calls frequency_1gram',
            'system calls tfidf_1gram', 'system calls hashing_1gram',
            'system calls frequency_2gram', 'system calls tfidf_2gram',
            'system calls hashing_2gram', 'system calls frequency_3gram',
            'system calls tfidf_3gram', 'system calls hashing_3gram',
            'system calls frequency_4gram', 'system calls tfidf_4gram',
            'system calls hashing_4gram', 'system calls frequency_5gram',
            'system calls tfidf_5gram', 'system calls hashing_5gram'
            ]
malwares=["delay", "disorder", "freeze", "hop", "mimic", "noise", "repeat", "spoof"]

In [33]:
def train_models_sequence(normal,ft, tw):
    ls = int(tw/10)
    X = normal[ft].tolist()
    y = np.ones(len(X))

    nu = 0.05
    outliers_fraction = 0.15
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3, random_state=42)

    if 'one hot' in ft:
        X_train = reshape_onehot(X_train, ls)
        X_val = reshape_onehot(X_val, ls)
    elif 'dict' in ft:
        X_train = reshape_dictindex(X_train, ls)
        X_val = reshape_dictindex(X_val, ls)

    X_train_pca = {}
    X_val_pca = {}

    pcas = {}
    onehot_clfs = {}
    pred = {}
    results = []
    classifiers = {
                    # "Robust covariance": EllipticEnvelope(contamination=outliers_fraction),
                    # "One-Class SVM": OneClassSVM(nu=outliers_fraction, kernel="rbf",gamma=0.1),
                    "SGD One-Class SVM": SGDOneClassSVM(nu=nu, shuffle=True, fit_intercept=True, random_state=42, tol=1e-4),
                    "Isolation Forest": IsolationForest(contamination=outliers_fraction,random_state=42),
                }

    for i in range(0,ls):
        pcas[i] = PCA(n_components=10)
        X_train_pca[i] = pcas[i].fit_transform(X_train[i])
        X_val_pca[i] = pcas[i].transform(X_val[i])

    for name in classifiers:
        t1  = time.time()
        clfs ={}
        clfs_pca = {}
        y_tr_pre = {}
        y_val_pre = {}
        y_tr_pca_pre = {}
        y_val_pca_pre = {}
        for i in range(0,ls):
            clfs[i] = copy.deepcopy(classifiers[name])
            clfs[i].fit(X_train[i])
            y_tr_pre[i] = clfs[i].predict(X_train[i])
            y_val_pre[i] = clfs[i].predict(X_val[i])
        y_tr_pre = pd.DataFrame(y_tr_pre)
        clfs[ls] = classifiers[name]
        clfs[ls].fit(y_tr_pre)
        t2  = time.time()

        y_val_pre = pd.DataFrame(y_val_pre)
        y_pred = clfs[ls].predict(y_val_pre)

        score = metrics.accuracy_score(y_val,y_pred)
        feature = ft
        pred['valid_'+  feature + '_' + name] = y_pred
        t = t2 -t1
        res = {}
        res['Model'] ='valid_' + feature + '_' + name
        res['Accuracy'] = score
        res['Training time'] = t
        results.append(res)
        onehot_clfs[feature + '_' + name] = clfs
        print('Model: {}, accuracy score: {}, training time is: {} seconds'.format(res['Model'], score, t))



        t1  = time.time()
        for i in range(0,ls):
            clfs_pca[i] = copy.deepcopy(classifiers[name])
            clfs_pca[i].fit(X_train_pca[i])
            y_tr_pca_pre[i] = clfs_pca[i].predict(X_train_pca[i])
            y_val_pca_pre[i] = clfs_pca[i].predict(X_val_pca[i])
        y_tr_pca_pre = pd.DataFrame(y_tr_pca_pre)
        clfs_pca[ls] = classifiers[name]
        clfs_pca[ls].fit(y_tr_pca_pre)
        t2  = time.time()

        y_val_pca_pre = pd.DataFrame(y_val_pca_pre)
        y_pred_pca = clfs[ls].predict(y_val_pca_pre)
        
        score = metrics.accuracy_score(y_val,y_pred_pca)

        feature =  ft + ' PCA'
        pred['valid_'+  feature + '_' + name] = y_pred
        t = t2 -t1
        res = {}
        res['Model'] ='valid_' + feature + '_' + name
        res['Accuracy'] = score
        res['Training time'] = t
        results.append(res)
        onehot_clfs[feature + '_' + name] = clfs_pca
        print('Model: {}, accuracy score: {}, training time is: {} seconds'.format(res['Model'], score, t))
    return onehot_clfs, pred, results, pcas

In [34]:
devices = ['pi3', 'pi4_2G', 'pi4_4G']
tws = [50, 60, 70]
for device in devices:
    for tw in tws:
        run(device, tw)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/git/IoT_Sensors_Security_Analysis/data/perf/encoded/t1/encoded_bowpi3_50.pkl'

In [ ]:
encoded_trace_df = pd.read_pickle(encodePath+'encoded_bow{}_{}.pkl'.format('pi4_2G', '50'))
one_hot = encoded_trace_df['one hot encoding']
lens = pd.DataFrame([len(i) for i in one_hot])
lens.describe()

,0
count,3321.000000
mean,151603.051491
std,27801.679020
min,14.000000
25%,148941.000000
50%,163632.000000
75%,166675.000000
max,173725.000000


In [ ]:
encoded_trace_df = pd.read_pickle(encodePath+'encoded_bow{}_{}.pkl'.format('pi4_2G', '60'))
one_hot = encoded_trace_df['one hot encoding']
lens = pd.DataFrame([len(i) for i in one_hot])
lens.describe()

,0
count,2766.000000
mean,181448.506146
std,32359.959890
min,4600.000000
25%,177720.500000
50%,195877.000000
75%,199687.000000
max,207901.000000


In [ ]:
encoded_trace_df = pd.read_pickle(encodePath+'encoded_bow{}_{}.pkl'.format('pi4_2G', '70'))
one_hot = encoded_trace_df['one hot encoding']
lens = pd.DataFrame([len(i) for i in one_hot])
lens.describe()

,0
count,2386.000000
mean,210659.577117
std,38037.244949
min,12060.000000
25%,203808.000000
50%,227928.500000
75%,232368.750000
max,242128.000000


In [ ]:
def padding(onehot, dw):
    padding = dw*3000
    if len(onehot) > padding:
        onehot = np.array(onehot[0:padding])
    else:
        onehot =np.pad(onehot, [(0, padding-len(onehot)), (0, 0)], mode='constant', constant_values=0)
    return onehot

In [ ]:
def reshape_onehot(onehot_list, dw):
    reshaped = {}
    for i in range(0, dw):
        reshaped[i] = []
    for onehot in onehot_list:
        onehot = padding(onehot, dw)
        for i in range(0, dw):
            start = 3000*i
            end = 3000*(i+1)
            reshaped[i].append(np.array(onehot[start:end]).reshape(-1))
    return reshaped

In [ ]:
def reshape_dictindex(dictindex, dw):
    reshaped = {}
    for i in range(0, dw):
        reshaped[i] = []
    padding = dw*3000
    for di in dictindex:
        if len(di) > padding:
            di = np.array(di[0:padding])
        else:
            di =np.pad(di, [(0, padding-len(di))], mode='constant', constant_values=0)
        for i in range(0, dw):
            start = 3000*i
            end = 3000*(i+1)
            reshaped[i].append(np.array(di[start:end]).reshape(-1))
    return reshaped

In [ ]:
def train_models_sequence(normal,ft, tw):
    X = normal[ft].tolist()
    y = np.ones(len(X))

    nu = 0.05
    outliers_fraction = 0.15
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3, random_state=42)

    if 'one hot' in ft:
        X_train = reshape_onehot(X_train, tw)
        X_val = reshape_onehot(X_val, tw)
    elif 'dict' in ft:
        X_train = reshape_dictindex(X_train, tw)
        X_val = reshape_dictindex(X_val, tw)

    X_train_pca = {}
    X_val_pca = {}

    pcas = {}
    onehot_clfs = {}
    pred = {}
    results = []
    classifiers = {
                    # "Robust covariance": EllipticEnvelope(contamination=outliers_fraction),
                    "One-Class SVM": OneClassSVM(nu=outliers_fraction, kernel="rbf",gamma=0.1),
                    "SGD One-Class SVM": SGDOneClassSVM(nu=nu, shuffle=True, fit_intercept=True, random_state=42, tol=1e-4),
                    "Isolation Forest": IsolationForest(contamination=outliers_fraction,random_state=42),
                }

    for i in range(0, tw):
        pcas[i] = PCA(n_components=100)
        X_train_pca[i] = pcas[i].fit_transform(X_train[i])
        X_val_pca[i] = pcas[i].transform(X_val[i])

    for name in classifiers:
        t1  = time.time()
        clfs ={}
        clfs_pca = {}
        y_tr_pre = {}
        y_val_pre = {}
        y_tr_pca_pre = {}
        y_val_pca_pre = {}
        par = tqdm.tqdm(total=tw,ncols=80)
        for i in range(0, tw):
            par.update(1)
            clfs[i] = copy.deepcopy(classifiers[name])
            clfs[i].fit(X_train[i])
            y_tr_pre[i] = clfs[i].predict(X_train[i])
            y_val_pre[i] = clfs[i].predict(X_val[i])
        par.close()
        y_tr_pre = pd.DataFrame(y_tr_pre)
        clfs[tw] = classifiers[name]
        clfs[tw].fit(y_tr_pre)
        t2  = time.time()

        y_val_pre = pd.DataFrame(y_val_pre)
        y_pred = clfs[tw].predict(y_val_pre)

        score = metrics.accuracy_score(y_val,y_pred)
        feature = ft
        pred['valid_'+  feature + '_' + name] = y_pred
        t = t2 -t1
        res = {}
        res['Model'] ='valid_' + feature + '_' + name
        res['Accuracy'] = score
        res['Training time'] = t
        results.append(res)
        onehot_clfs[feature + '_' + name] = clfs
        print('Model: {}, accuracy score: {}, training time is: {} seconds'.format(res['Model'], score, t))
        
        t1  = time.time()
        for i in range(0,tw):
            clfs_pca[i] = copy.deepcopy(classifiers[name])
            clfs_pca[i].fit(X_train_pca[i])
            y_tr_pca_pre[i] = clfs_pca[i].predict(X_train_pca[i])
            y_val_pca_pre[i] = clfs_pca[i].predict(X_val_pca[i])
        y_tr_pca_pre = pd.DataFrame(y_tr_pca_pre)
        clfs_pca[tw] = classifiers[name]
        clfs_pca[tw].fit(y_tr_pca_pre)
        t2  = time.time()

        y_val_pca_pre = pd.DataFrame(y_val_pca_pre)
        y_pred_pca = clfs[tw].predict(y_val_pca_pre)
        
        score = metrics.accuracy_score(y_val,y_pred_pca)

        feature =  ft + ' PCA'
        pred['valid_'+  feature + '_' + name] = y_pred
        t = t2 -t1
        res = {}
        res['Model'] ='valid_' + feature + '_' + name
        res['Accuracy'] = score
        res['Training time'] = t
        results.append(res)
        onehot_clfs[feature + '_' + name] = clfs_pca
        print('Model: {}, accuracy score: {}, training time is: {} seconds'.format(res['Model'], score, t))
    return onehot_clfs, pred, results, pcas

In [ ]:
def test__sequence(encoded_trace_df, malware, ft, tw, onehot_clfs, pcas):
    feature = ft

    dfs = encoded_trace_df[encoded_trace_df.maltype==malware]
    X_test = dfs[feature].tolist()
    
    y_test = np.ones(len(X_test))
    y_test_2 = [1 if i==0 else -1 for i in y_test]

    if 'one hot' in ft:
        X_test = reshape_onehot(X_test, tw)
    elif 'dict' in ft:
        X_test = reshape_dictindex(X_test, tw)

    X_test_pca = {}
    y_test_pre = {}
    y_test_pca_pre = {}

    pred_test = {}
    results_test = []
    classifiers = [
        #  "Robust covariance",
                 "One-Class SVM",
                "SGD One-Class SVM",
                "Isolation Forest"]

    for i in range(0,tw):
        X_test_pca[i] = pcas[i].transform(X_test[i])

    for name in classifiers:
        y_test_pre = {}
        y_test_pca_pre = {}
        t1  = time.time()
        
        feature = ft
        modelName = feature + '_' + name
        clfs = onehot_clfs[modelName]
        for i in range(0,tw):
            y_test_pre[i] = clfs[i].predict(X_test[i])
        y_test_pre = pd.DataFrame(y_test_pre)
        y_test_pre = clfs[tw].predict(y_test_pre)
        t2  = time.time()

        score = metrics.accuracy_score(y_test_2, y_test_pre)

        pred_test[malware + '_'+  feature + '_' + name] = y_test_pre
        t = t2 -t1
        res = {}
        res['Model'] =malware + '_'+  feature + '_' + name
        res['Accuracy'] = score
        res['test time'] = t
        results_test.append(res)
        print('Model: {}, accuracy score: {}, test time is: {} seconds'.format(res['Model'], score, t))

        feature = ft + ' PCA'
        modelName = feature + '_' + name
        clfs_pca = onehot_clfs[modelName]
        t1  = time.time()
        for i in range(0,tw):
            y_test_pca_pre[i] = clfs_pca[i].predict(X_test_pca[i])
            
        y_test_pca_pre = pd.DataFrame(y_test_pca_pre)
        y_test_pca_pre = clfs[tw].predict(y_test_pca_pre)
        t2  = time.time()  
        
        score = metrics.accuracy_score(y_test_2,y_test_pca_pre)
        
        pred_test[malware + '_'+  feature + '_' + name] = y_test_pre
        t = t2 -t1
        res = {}
        res['Model'] =malware + '_'+  feature + '_' + name
        res['Accuracy'] = score
        res['test time'] = t
        results_test.append(res)
        print('Model: {}, accuracy score: {}, test time is: {} seconds'.format(res['Model'], score, t))
    return results_test, pred_test

In [ ]:
def run__sequence(device, tw, features):
    resultsPath = 'D:/git/IoT_Sensors_Security_Analysis/results/{}/tw_{}_turn_1/'.format(device, tw)
    encoded_trace_df = pd.read_pickle(encodePath+'encoded_bow{}_{}.pkl'.format(device, tw))
    normal = encoded_trace_df[encoded_trace_df.maltype=='normal']
    resultsdict = dict()
    predsdict = dict()
    classifiersdict = dict()
    pcas = dict()
    for ft in features:
        #train stage
        onehot_clfs, pred, results, pca = train_models_sequence(normal,ft, tw)        
        resultsdict[ft+'_validation'] = results
        predsdict[ft+'_validation'] = pred
        classifiersdict[ft] = onehot_clfs
        pcas[ft] = pca
        # testing stage
        for malware in malwares:
            results_test, pred_test = test__sequence(encoded_trace_df, malware, ft, tw, onehot_clfs, pca)
            resultsdict[malware +'_' + ft] = results_test
            predsdict[malware +'_' + ft] = pred_test
            
    loc=open(resultsPath+'sequence_classifiers.pk','wb')
    pickle.dump(classifiersdict,loc)
    loc=open(resultsPath+'sequence_results.pk','wb')
    pickle.dump(resultsdict,loc)
    loc=open(resultsPath+'sequence_preds.pk','wb')
    pickle.dump(predsdict,loc)   
    loc=open(resultsPath+'sequence_pcas.pk','wb')
    pickle.dump(pcas,loc)   
    rd = []
    for rs in resultsdict:
        for r in resultsdict[rs]:        
            rd.append(r)
    rd = pd.DataFrame(rd)
    md = [i.split('_') for i in rd['Model']]
    md = pd.DataFrame(md)
    md.columns  = ['Dataset','Features','Model']
    nrd=pd.DataFrame([md['Dataset'],md['Features'], md['Model'], rd['Accuracy']]).transpose()
    nrd.to_csv(resultsPath+'sequence_results.csv',index=None)
    return resultsdict

In [ ]:
sft = ['one hot encoding', 'dict index encoding']
malwares=["delay", "disorder", "freeze", "hop", "mimic", "noise", "repeat", "spoof"]

In [ ]:
devices = ['pi3', 'pi4_2G', 'pi4_4G']
tws = [50, 60, 70]
for device in devices:
    for tw in tws:
        run__sequence(device, tw, sft)

100%|███████████████████████████████████████████| 50/50 [15:30<00:00, 18.60s/it]


Model: valid_one hot encoding_One-Class SVM, accuracy score: 0.33613445378151263, training time is: 930.1099662780762 seconds
Model: valid_one hot encoding PCA_One-Class SVM, accuracy score: 0.0, training time is: 0.6041457653045654 seconds


100%|███████████████████████████████████████████| 50/50 [00:11<00:00,  4.24it/s]


Model: valid_one hot encoding_SGD One-Class SVM, accuracy score: 1.0, training time is: 11.802725553512573 seconds
Model: valid_one hot encoding PCA_SGD One-Class SVM, accuracy score: 1.0, training time is: 0.050011634826660156 seconds


100%|███████████████████████████████████████████| 50/50 [17:09<00:00, 20.59s/it]


...

Model: valid_dict index encoding_One-Class SVM, accuracy score: 0.0, training time is: 14.952856302261353 seconds
Model: valid_dict index encoding PCA_One-Class SVM, accuracy score: 0.0, training time is: 0.6741526126861572 seconds


100%|██████████████████████████████████████████| 70/70 [00:00<00:00, 114.27it/s]


Model: valid_dict index encoding_SGD One-Class SVM, accuracy score: 1.0, training time is: 0.6165862083435059 seconds
Model: valid_dict index encoding PCA_SGD One-Class SVM, accuracy score: 1.0, training time is: 0.14403128623962402 seconds


100%|███████████████████████████████████████████| 70/70 [01:16<00:00,  1.09s/it]


Model: valid_dict index encoding_Isolation Forest, accuracy score: 0.6829268292682927, training time is: 76.35523176193237 seconds
Model: valid_dict index encoding PCA_Isolation Forest, accuracy score: 1.0, training time is: 16.952894926071167 seconds
Model: delay_dict index encoding_One-Class SVM, accuracy score: 1.0, test time is: 11.45387315750122 seconds
Model: delay_dict index encoding PCA_One-Class SVM, accuracy score: 1.0, test time is: 0.38408684730529785 seconds
Model: delay_dict index encoding_SGD One-Class SVM, accuracy score: 0.04721030042918455, test time is: 0.16602802276611328 seconds
Model: delay_dict index encoding PCA_SGD One-Class SVM, accuracy score: 0.0, test time is: 0.014003753662109375 seconds
Model: delay_dict index encoding_Isolation Forest, accuracy score: 0.34334763948497854, test time is: 18.152623176574707 seconds
Model: delay_dict index encoding PCA_Isolation Forest, accuracy score: 0.0, test time is: 3.288759708404541 seconds
Model: disorder_dict index e

In [ ]:
# device = 'pi3'
# tw = 50
# resultsPath = 'D:/git/IoT_Sensors_Security_Analysis/results/{}/tw_{}_turn_1/'.format(device, tw)
# loc=open(resultsPath+'results.pk','rb')
# results =  pickle.load(loc)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM
from sklearn.svm import OneClassSVM
from sklearn import metrics

import time
import re
import ast
import os,sys
import pickle


In [2]:

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [53]:
def train_models(feature, normal):
    X = normal[feature].tolist()
    # if len(X[0]) > 100:
    #     X = [h[0:100] for h in X]
    # if 'frequency' not in feature:
    #     X = [[i*1000 for i in h] for h in X]

    y = [1 for i in range(0,len(X))]
    # h = .02  # step size in the mesh
    outliers_fraction = 0.05
    # nu = 0.01
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3, shuffle=False)
    y_val = [1 for i in range(0,len(X_val))]
    
    result = []
    pred = dict()
    classifiers = {}
    classifiers = {
            "Robust covariance": EllipticEnvelope(contamination=outliers_fraction , support_fraction=0.5),
            "One-Class SVM": OneClassSVM(cache_size=200, gamma='scale', kernel='rbf',nu=0.05,  shrinking=True, tol=0.001,verbose=False),
            "SGD One-Class SVM": SGDOneClassSVM(nu=outliers_fraction, shuffle=True, fit_intercept=True, random_state=42, tol=1e-4),
            "Isolation Forest": IsolationForest(contamination=outliers_fraction,random_state=42),
        }
    for name in classifiers:
        clf = classifiers[name]
        t1 =time.time()
        res = dict()
        print( feature + '_' + name)
        try:
            clf.fit(X_train)
            t2 =time.time()        
            y_pred = clf.predict(X_val)
            score = metrics.accuracy_score(y_val,y_pred)
        except:
            t2 =time.time()
            y_pred = []
            score = 0
        
        pred['valid_'+  feature + '_' + name] = y_pred
        t = t2 -t1
        res['Model'] ='valid_' + feature + '_' + name
        res['Accuracy'] = score
        res['Training time'] = t
        result.append(res)
        print('Model: {}, accuracy score: {}, training time is: {} seconds'.format(res['Model'], score, t))
    return classifiers, result, pred

def test_models(encoded_trace_df, malware, feature, classifiers):
    dfs = encoded_trace_df[encoded_trace_df.maltype==malware]
    X_test = dfs[feature].tolist()
    # if len(X_test[0]) > 100:        
    #     X_test = [h[0:100] for h in X_test]
    # if 'frequency' not in feature:
    #     X_test = [[i*1000 for i in h] for h in X_test]
    # y_test = np.ones(len(X_test))
    # y_test = [1 if i==0 else -1 for i in y_test]
    y_test = [-1 for i in range(0,len(X_test))]
    
    result = []
    pred = dict()    
    for name in classifiers:
        res = dict()
        clf = classifiers[name]
        t1 =time.time()
        try:
            y_pred = clf.predict(X_test)
            t2 =time.time()
            score = metrics.accuracy_score(y_test, y_pred)
        except:
            t2 =time.time()
            y_pred = []
            score = 0
        t = t2 -t1
        pred[malware +'_' + feature + '_' + name] = y_pred
        res['Model'] = malware +'_' + feature + '_' + name
        res['Accuracy'] = score
        res['Testing time'] = t
        result.append(res)
        # print('Model: {}, accuracy score: {}, testing time is: {} seconds'.format( res['Model'], score, t))
    return  result, pred

def run(device, tw):
    rootPath = 'd:/mt_data/1126_withrw/'
    resultsPath = 'd:/mt_data/1126_withrw/'
    dataPath = rootPath+'encoded/t1/' 
    resultsdict = dict()
    predsdict = dict()
    classifiersdict = dict()

    for feature in features:
        #train stage
        # read data from file
        tsv_name = dataPath+'encoded_bow{}_{}_{}.csv'.format(device, tw, feature)
        encoded_trace_df = pd.read_csv(tsv_name, sep='\t')
        ft = [ast.literal_eval(i) for i in encoded_trace_df[feature]]
        encoded_trace_df[feature] = ft

        normal = encoded_trace_df[encoded_trace_df.maltype=='normal']
        clfs, results, preds = train_models(feature, normal)
        
        resultsdict[feature+'_validation'] = results
        predsdict[feature+'_validation'] = preds
        classifiersdict[feature] = clfs
 
        # testing stage
        for malware in malwares:
            results, preds = test_models(encoded_trace_df, malware, feature, clfs)
            resultsdict[malware +'_' + feature] = results
            predsdict[malware +'_' + feature] = preds
            
    loc=open(resultsPath+'classifiers_{}_{}.pk'.format(device, tw),'wb')
    pickle.dump(classifiersdict,loc)
    loc=open(resultsPath+'results_{}_{}.pk'.format(device, tw),'wb')
    pickle.dump(resultsdict,loc)
    loc=open(resultsPath+'preds_{}_{}.pk'.format(device, tw),'wb')
    pickle.dump(predsdict,loc)   

    rd = []
    for rs in resultsdict:
        for r in resultsdict[rs]:
                rd.append(r)

    rd = pd.DataFrame(rd)
    md = [i.split('_') for i in rd['Model']]
    md = pd.DataFrame(md)
    md.columns  = ['Dataset','Features','Ngram','Model']
    nrd=pd.DataFrame([md['Dataset'],md['Features'],md['Ngram'], md['Model'], rd['Accuracy']]).transpose()
    nrd.to_csv(resultsPath+'results_{}_{}.csv'.format(device, tw),index=None)

In [33]:
from sklearn.decomposition import PCA

In [42]:
feature = 'system calls frequency_1gram'

In [43]:
normal = encoded_trace_df[encoded_trace_df.maltype=='normal'][feature].tolist() 
delay = encoded_trace_df[encoded_trace_df.maltype=='delay'][feature].tolist()
len(normal[0])

17

In [36]:
X_train, X_val = train_test_split(normal, test_size=.3) 
    
cl = OneClassSVM(cache_size=200, max_iter=-1, gamma='scale', kernel='rbf',nu=0.1,  shrinking=False, tol=0.001,verbose=False)
cl.fit(X_train)
pr = cl.predict(X_val)
tr = np.ones(len(X_val))

print("acc for normal is {}".format(metrics.accuracy_score(pr,tr)))

pr = cl.predict(delay)
tr = np.ones(len(delay))
print("acc for delay is {}".format(1 - metrics.accuracy_score(pr,tr)))

acc for normal is 0.9166666666666666
acc for delay is 0.07499999999999996


In [48]:
m = max([max(i) for i in (normal+ delay)])

In [49]:
normal = [[j/m for j in i] for i in normal]
delay = [[j/m for j in i] for i in delay]

In [50]:
X_train, X_val = train_test_split(normal, test_size=.1, random_state=42) 
    
cl = OneClassSVM(cache_size=200, max_iter=-1, gamma='scale', kernel='rbf',nu=0.1,  shrinking=False, tol=0.001,verbose=False)
cl.fit(X_train)
pr = cl.predict(X_val)
tr = np.ones(len(X_val))

print("acc for normal is {}".format(metrics.accuracy_score(pr,tr)))

pr = cl.predict(delay)
tr = np.ones(len(delay))
print("acc for delay is {}".format(1 - metrics.accuracy_score(pr,tr)))

acc for normal is 0.9166666666666666
acc for delay is 0.07499999999999996


In [37]:
pca = PCA(n_components=10)
pca.fit(normal+ delay)
normal = pca.transform(normal)
delay = pca.transform(delay)

In [38]:
X_train, X_val = train_test_split(normal, test_size=.1, random_state=42) 
    
cl = OneClassSVM(cache_size=200, max_iter=-1, gamma='scale', kernel='rbf',nu=0.1,  shrinking=False, tol=0.001,verbose=False)
cl.fit(X_train)
pr = cl.predict(X_val)
tr = np.ones(len(X_val))

print("acc for normal is {}".format(metrics.accuracy_score(pr,tr)))

pr = cl.predict(delay)
tr = np.ones(len(delay))
print("acc for delay is {}".format(1 - metrics.accuracy_score(pr,tr)))

acc for normal is 0.9444444444444444
acc for delay is 0.7027777777777777


In [9]:
rootPath = 'd:/mt_data/1126_withrw/'
resultsPath = 'd:/mt_data/1126_withrw/'
dataPath = rootPath+'encoded/t1/' 
resultsdict = dict()
predsdict = dict()
classifiersdict = dict()
pca = PCA(n_components=2)
for feature in fts:
    #train stage
    # read data from file
    tsv_name = dataPath+'encoded_bow{}_{}_{}.csv'.format(device, tw, feature)
    encoded_trace_df = pd.read_csv(tsv_name, sep='\t')
    ft = [ast.literal_eval(i) for i in encoded_trace_df[feature]]
    encoded_trace_df[feature] = ft
    normal = []
    
    normal = encoded_trace_df[encoded_trace_df.maltype=='normal'][feature].tolist() 
    delay = encoded_trace_df[encoded_trace_df.maltype=='delay'][feature].tolist()
    pca.fit(normal+ delay)
    normal = pca.transform(normal)
    delay = pca.transform(delay)
    # if 'frequency' in feature:
    # normal = [h[0:100] for h in normal]
    X_train, X_val = train_test_split(normal, test_size=.1, random_state=42) 
    
    cl = OneClassSVM(cache_size=200, max_iter=-1, gamma='scale', kernel='rbf',nu=0.1,  shrinking=False, tol=0.001,verbose=False)
    cl.fit(X_train)
    
    # if 'frequency' in feature:
    # delay = [h[0:100] for h in delay]
    pr = cl.predict(delay)
    tr = np.ones(len(delay))
    print(1 - metrics.accuracy_score(pr,tr))
    # print(cl.decision_function(delay[0:9]))
    pr = cl.predict(X_val)
    tr = np.ones(len(X_val))
    print(metrics.accuracy_score(pr,tr))

0.7055555555555555
0.9444444444444444


In [61]:
if __name__ == "__main__":
    features = [#'one hot encoding', 'dict index encoding',
                # 'system calls dependency graph', 
                'system calls frequency_1gram',
                'system calls tfidf_1gram', 
                'system calls hashing_1gram',
                'system calls frequency_2gram', 
                'system calls tfidf_2gram',
                # 'system calls hashing_2gram',
                'system calls frequency_3gram',
                'system calls tfidf_3gram',
                # 'system calls hashing_3gram',
                # 'system calls frequency_4gram', 
                # 'system calls tfidf_4gram',
                # 'system calls hashing_4gram', 
                # 'system calls frequency_5gram',
                # 'system calls tfidf_5gram', 
                # 'system calls hashing_5gram',
                'system calls frequency_2gram-pcas', 
                'system calls tfidf_2gram-pcas',
                'system calls frequency_3gram-pcas',
                'system calls tfidf_3gram-pcas', 
                'system calls frequency_4gram-pcas', 
                'system calls tfidf_4gram-pcas',
                'system calls frequency_5gram-pcas',
                'system calls tfidf_5gram-pcas',
                'system calls frequency_1gram-scaled', 
                'system calls tfidf_1gram-scaled',                
                'system calls frequency_2gram-scaled', 
                'system calls tfidf_2gram-scaled',
                'system calls frequency_3gram-scaled',
                'system calls tfidf_3gram-scaled', 
                # 'system calls frequency_4gram-scaled', 
                # 'system calls tfidf_4gram-scaled',
                # 'system calls frequency_5gram-scaled',
                # 'system calls tfidf_5gram-scaled'
                ]

    malwares=["delay", "disorder", "freeze", "hop", "mimic", "noise", "repeat", "spoof"]
    devices =  ['pi3', 'pi4_2G', 'pi4_4G']
    tw =  60
    for device in devices:
        run(device, tw)

system calls frequency_1gram_Robust covariance
Model: valid_system calls frequency_1gram_Robust covariance, accuracy score: 0.9629629629629629, training time is: 0.021004676818847656 seconds
system calls frequency_1gram_One-Class SVM
Model: valid_system calls frequency_1gram_One-Class SVM, accuracy score: 0.9444444444444444, training time is: 0.0010006427764892578 seconds
system calls frequency_1gram_SGD One-Class SVM
Model: valid_system calls frequency_1gram_SGD One-Class SVM, accuracy score: 1.0, training time is: 0.00099945068359375 seconds
system calls frequency_1gram_Isolation Forest
Model: valid_system calls frequency_1gram_Isolation Forest, accuracy score: 0.9722222222222222, training time is: 0.11532163619995117 seconds
system calls tfidf_1gram_Robust covariance
Model: valid_system calls tfidf_1gram_Robust covariance, accuracy score: 0.9537037037037037, training time is: 0.020005464553833008 seconds
system calls tfidf_1gram_One-Class SVM
Model: valid_system calls tfidf_1gram_On

In [71]:
resultsPath = 'f:/temp/' 

In [92]:
loc = open(resultsPath+'preds_pi4_4G_60.pk','rb')
preds = pickle.load(loc)

In [93]:
preds

{'system calls frequency_1gram_validation': {'valid_system calls frequency_1gram_Robust covariance': array([-1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
          1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,
          1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1, -1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,  1,
          1,  1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  

In [64]:
import psutil as ps

In [75]:
(ps.Process(os.getpid()).memory_full_info().rss)/(pow(1024,3))

0.2259674072265625

In [78]:
rd = []
for rs in res:
    for r in res[rs]:
            rd.append(r)

In [79]:
rd

[{'Model': 'valid_system calls frequency_1gram_Robust covariance',
  'Accuracy': 0.8966666666666666,
  'Training time': 0.03200721740722656},
 {'Model': 'valid_system calls frequency_1gram_One-Class SVM',
  'Accuracy': 0.9033333333333333,
  'Training time': 0.00099945068359375},
 {'Model': 'valid_system calls frequency_1gram_SGD One-Class SVM',
  'Accuracy': 1.0,
  'Training time': 0.0010006427764892578},
 {'Model': 'valid_system calls frequency_1gram_Isolation Forest',
  'Accuracy': 0.8833333333333333,
  'Training time': 0.10802388191223145},
 {'Model': 'delay_system calls frequency_1gram_Robust covariance',
  'Accuracy': 0.1,
  'Testing time': 0.0010008811950683594},
 {'Model': 'delay_system calls frequency_1gram_One-Class SVM',
  'Accuracy': 0.13333333333333333,
  'Testing time': 0.00099945068359375},
 {'Model': 'delay_system calls frequency_1gram_SGD One-Class SVM',
  'Accuracy': 0.0,
  'Testing time': 0.0010006427764892578},
 {'Model': 'delay_system calls frequency_1gram_Isolation

In [80]:
rd = pd.DataFrame(rd)
    

In [89]:
rd

,Model,Accuracy,Training time,Testing time
0,valid_system calls frequency_1gram_Robust cova...,0.896667,0.032007,NaN
1,valid_system calls frequency_1gram_One-Class SVM,0.903333,0.000999,NaN
2,valid_system calls frequency_1gram_SGD One-Cla...,1.000000,0.001001,NaN
3,valid_system calls frequency_1gram_Isolation F...,0.883333,0.108024,NaN
4,delay_system calls frequency_1gram_Robust cova...,0.100000,NaN,0.001001
...,...,...,...,...
607,repeat_system calls tfidf_5gram-pcas_Isolation...,0.156667,NaN,0.031007
608,spoof_system calls tfidf_5gram-pcas_Robust cov...,1.000000,NaN,0.003001
609,spoof_system calls tfidf_5gram-pcas_One-Class SVM,1.000000,NaN,0.003000
610,spoof_system calls tfidf_5gram-pcas_SGD One-Cl...,1.000000,NaN,0.001000


In [90]:
md = [i.split('_') for i in rd['Model']]
md = pd.DataFrame(md)
md.columns  = ['Dataset','Features','Ngram','Model']


In [91]:
md

,Dataset,Features,Ngram,Model
0,valid,system calls frequency,1gram,Robust covariance
1,valid,system calls frequency,1gram,One-Class SVM
2,valid,system calls frequency,1gram,SGD One-Class SVM
3,valid,system calls frequency,1gram,Isolation Forest
4,delay,system calls frequency,1gram,Robust covariance
...,...,...,...,...
607,repeat,system calls tfidf,5gram-pcas,Isolation Forest
608,spoof,system calls tfidf,5gram-pcas,Robust covariance
609,spoof,system calls tfidf,5gram-pcas,One-Class SVM
610,spoof,system calls tfidf,5gram-pcas,SGD One-Class SVM


In [2]:
@profile(precision=4,stream=open('memory_profiler.log','w+'))
def test1():
    c=0
    for item in range(0, 100000):
        c+=1

In [4]:
if __name__=='__main__':
    test1()

ERROR: Could not find file C:\Users\luke-\AppData\Local\Temp/ipykernel_10396/1408896442.py
